# Treinamento do Mecanismo de Recomendação

*Dataset: https://www.kaggle.com/datasets/ylchang/coffee-shop-sample-data-1113*

In [51]:
# Bibliotecas
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

# Leitura do dataset

In [52]:
# Recibos de venda
sales_reciepts = pd.read_csv("dataset/201904 sales reciepts.csv")
sales_reciepts.head() # Exibir dados

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [53]:
# Produtos
product = pd.read_csv("dataset/product.csv")
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


# Manipulação dos dados

## Filtragem dos dados
Filtrar as colunas necessárias nos dataframes.

In [54]:
sales_reciepts = sales_reciepts[[
    "transaction_id", "transaction_date", "sales_outlet_id", "customer_id", "product_id", "quantity"]]
sales_reciepts.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity
0,7,2019-04-01,3,558,52,1
1,11,2019-04-01,3,781,27,2
2,19,2019-04-01,3,788,46,2
3,32,2019-04-01,3,683,23,2
4,33,2019-04-01,3,99,34,1


In [55]:
product = product[[
    "product_id", "product_category", "product"
]]
product.head()

,product_id,product_category,product
0,1,Coffee beans,Brazilian - Organic
1,2,Coffee beans,Our Old Time Diner Blend
2,3,Coffee beans,Espresso Roast
3,4,Coffee beans,Primo Espresso Roast
4,5,Coffee beans,Columbian Medium Roast


## Merge Data
Mescla os dados dos dataframes.

In [56]:
# Mesclar os dados do dataset
dataset = pd.merge(
    sales_reciepts, product, # dataframes
    on="product_id", # coluna correspondente
    how="left" # "LEFT JOIN"
    )
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


## Remover tamanhos
Alguns produtos possuem no nome seus tamanhos, como "lg", "rg", "small", etc.

In [57]:
# Filtrar os dados que possuem o produto "Dark chocolate" para obter os tamanhos (lg, rg, default)
dataset[dataset["product"].str.contains("Dark chocolate")]["product"].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [58]:
# Quantidade de produtos (inclusive variações de tamanhos)
dataset["product"].nunique()

80

In [59]:
# Lista de produtos com variações de tamanho
print(dataset["product"].unique())

['Traditional Blend Chai Rg' 'Brazilian Lg' 'Serenity Green Tea Rg'
 'Our Old Time Diner Blend Rg' 'Jamaican Coffee River Sm' 'Ethiopia Rg'
 'English Breakfast Lg' 'Sustainably Grown Organic Rg' 'Earl Grey Lg'
 'Jamaican Coffee River Rg' 'Serenity Green Tea Lg' 'Brazilian Sm'
 'English Breakfast Rg' 'Traditional Blend Chai Lg' 'Cappuccino'
 'Espresso shot' 'Cappuccino Lg' 'Latte' 'Earl Grey Rg'
 'Dark chocolate Lg' 'Columbian Medium Roast Sm' 'Oatmeal Scone'
 'Morning Sunrise Chai Lg' 'Morning Sunrise Chai Rg' 'Peppermint Lg'
 'Jumbo Savory Scone' 'Lemon Grass Lg' 'Sustainably Grown Organic Lg'
 'Dark chocolate Rg' 'Lemon Grass Rg' 'Ethiopia Sm' 'Latte Rg'
 'Our Old Time Diner Blend Sm' 'Chocolate Chip Biscotti'
 'Columbian Medium Roast Rg' 'Ethiopia Lg' 'Brazilian Rg'
 'Columbian Medium Roast Lg' 'Spicy Eye Opener Chai Rg' 'Ginger Biscotti'
 'Our Old Time Diner Blend Lg' 'Chocolate Croissant'
 'Jamaican Coffee River Lg' 'Hazelnut Biscotti' 'Spicy Eye Opener Chai Lg'
 'Cranberry Scone'

In [60]:
# Remover os tamanhos do nome do produto
sizes = ["Rg", "Sm", "Lg"] # Lista de tamanhos

for s in sizes:
    dataset["product"] = dataset["product"].str.replace(
        f" {s}", "" # remover tamanho
    )

In [61]:
# Quantidade de produtos (sem contar variações de tamanho)
dataset["product"].nunique()

45

In [62]:
# Lista de produtos sem variações de tamanho
print(dataset["product"].unique())

['Traditional Blend Chai' 'Brazilian' 'Serenity Green Tea'
 'Our Old Time Diner Blend' 'Jamaican Coffee River' 'Ethiopia'
 'English Breakfast' 'Sustainably Grown Organic' 'Earl Grey' 'Cappuccino'
 'Espresso shot' 'Latte' 'Dark chocolate' 'Columbian Medium Roast'
 'Oatmeal Scone' 'Morning Sunrise Chai' 'Peppermint' 'Jumbo Savory Scone'
 'Lemon Grass' 'Chocolate Chip Biscotti' 'Spicy Eye Opener Chai'
 'Ginger Biscotti' 'Chocolate Croissant' 'Hazelnut Biscotti'
 'Cranberry Scone' 'Scottish Cream Scone ' 'Croissant' 'Almond Croissant'
 'Ginger Scone' 'Ouro Brasileiro shot' 'Organic Decaf Blend'
 'Chocolate syrup' 'Hazelnut syrup' 'Carmel syrup'
 'Sugar Free Vanilla syrup' 'Jamacian Coffee River'
 'Guatemalan Sustainably Grown' 'Civet Cat' 'Chili Mayan'
 'Primo Espresso Roast' 'Brazilian - Organic' 'I Need My Bean! Diner mug'
 'Espresso Roast' 'I Need My Bean! T-shirt' 'I Need My Bean! Latte cup']


## Filtrar produtos que serão realmente vendidos

In [63]:
# Produtos que serão vendidos
products_to_take = [
    'Cappuccino', 'Latte', 'Espresso shot', 'Dark chocolate', 
    'Sugar Free Vanilla syrup', 'Chocolate syrup', 'Carmel syrup', 
    'Hazelnut syrup', 'Ginger Scone', 'Chocolate Croissant', 
    'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',
    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 
    'Chocolate Chip Biscotti', 'Ginger Biscotti'
    ]

In [64]:
dataset.size # Tamanho antes da filtragem

399152

In [65]:
# Filtrar os produtos
dataset = dataset[
    dataset['product'].isin(products_to_take) # mantém os dados com produtos que estão na lista
]
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino


In [66]:
dataset.size  # Tamanho depois da filtragem

127432

## Listar Categorias

In [67]:
# Listar produtos e suas categorias
products_categories = dataset[["product", "product_category"]].drop_duplicates().reset_index(drop=True)
# Remove as duplicatas e reseta o index

display(products_categories)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Oatmeal Scone,Bakery
5,Jumbo Savory Scone,Bakery
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


## Limpeza das transações
Concatenar o `transaction_id` com `customer_id` para criar um novo id de transação e evitar confusões no modelo.

In [68]:
# Criar nova coluna com o id da transação e do cliente
dataset["transaction"] = dataset["transaction_id"].astype(str) + "_" + dataset["customer_id"].astype(str)
# Converte conteúdo das colunas em string e concatena-os

dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


Remover usuários que compraram somente 1 item (esses casos são desnecessários pro modelo de recomendação).

In [69]:
# Criar tabela com a quantidade de produtos por transação
num_of_items_for_each_transaction = dataset["transaction"].value_counts().reset_index()

num_of_items_for_each_transaction

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24
...,...,...
8381,501_659,1
8382,202_437,1
8383,224_788,1
8384,226_533,1


In [70]:
# Quantidade de transações com apenas 1 produto
num_of_items_for_each_transaction[num_of_items_for_each_transaction["count"] == 1]

,transaction,count
2646,221_8481,1
2647,97_8311,1
2648,510_288,1
2649,513_486,1
2650,516_206,1
...,...,...
8381,501_659,1
8382,202_437,1
8383,224_788,1
8384,226_533,1


In [71]:
# Filtrar as transações com mais de 1 item
# Armazenar os IDs das transações válidas em uma lista
valid_transactions = num_of_items_for_each_transaction[
    (num_of_items_for_each_transaction["count"] > 1)]["transaction"].tolist()

print(valid_transactions)
print("Quantidade de transações válidas:", len(valid_transactions))

['209_0', '206_0', '204_0', '208_0', '203_0', '207_0', '205_0', '151_0', '301_0', '323_0', '42_0', '202_0', '9_0', '76_0', '213_0', '307_0', '274_0', '46_0', '149_0', '51_0', '195_0', '475_0', '197_0', '298_0', '6_0', '242_0', '329_0', '370_0', '140_0', '319_0', '344_0', '54_0', '310_0', '223_0', '325_0', '132_0', '118_0', '317_0', '255_0', '102_0', '172_0', '59_0', '10_0', '320_0', '75_0', '313_0', '278_0', '146_0', '135_0', '314_0', '3_0', '282_0', '36_0', '97_0', '216_0', '228_0', '266_0', '122_0', '38_0', '334_0', '315_0', '61_0', '291_0', '352_0', '284_0', '144_0', '251_0', '211_0', '234_0', '345_0', '239_0', '356_0', '253_0', '187_0', '377_0', '43_0', '30_0', '289_0', '171_0', '421_0', '280_0', '311_0', '152_0', '45_0', '410_0', '260_0', '373_0', '359_0', '143_0', '411_0', '268_0', '306_0', '85_0', '164_0', '262_0', '170_0', '333_0', '177_0', '167_0', '190_0', '58_0', '235_0', '227_0', '295_0', '327_0', '82_0', '264_0', '32_0', '360_0', '331_0', '343_0', '466_0', '324_0', '287_0'

In [72]:
# Filtrar as transações do dataset com a lista de transações válidas
dataset = dataset[dataset["transaction"].isin(valid_transactions)]

# Tendências de Produtos

## Mecanismo de Recomendação por Popularidade

**Produtos/categorias mais populares:** Usados para recomendações para novos clientes que não possuem nenhum produto adicionado ao carrinho.

In [73]:
# Categorias mais populares
dataset["product_category"].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

In [74]:
# Produtos mais populares
dataset["product"].value_counts()

product
Cappuccino                  1290
Latte                       1256
Dark chocolate               969
Chocolate Croissant          636
Espresso shot                628
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Ginger Scone                 417
Jumbo Savory Scone           357
Croissant                    355
Chocolate Chip Biscotti      352
Cranberry Scone              350
Almond Croissant             347
Hazelnut Biscotti            338
Oatmeal Scone                334
Ginger Biscotti              314
Name: count, dtype: int64

In [75]:
# 
product_recommendation = dataset.groupby(
    ["product", "product_category"] # agrupar produto por categoria
    ).count().reset_index()

# Usar apenas as colunas product, product_category e product_id
product_recommendation = product_recommendation[
    ["product", "product_category", "product_id"]
]
# Renomear colunas
product_recommendation = product_recommendation.rename(columns={
    "transaction_id": "number_of_transactions"})

product_recommendation

,product,product_category,product_id
0,Almond Croissant,Bakery,347
1,Cappuccino,Coffee,1290
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,352
4,Chocolate Croissant,Bakery,636
5,Chocolate syrup,Flavours,568
6,Cranberry Scone,Bakery,350
7,Croissant,Bakery,355
8,Dark chocolate,Drinking Chocolate,947
9,Dark chocolate,Packaged Chocolate,22


In [76]:
# Exportar recomendações para novos clientes em CSV
product_recommendation.to_csv(
    "api/recommendation_objects/popularity_recommendation.csv", # arquivo de destino
    index=False # não incluir o índice
)

# Mecanismo de recomendação Apriori

In [77]:
# Dados de treinamento
# Agrupa as transações e os produtos e adiciona um contador pros produtos
train_basket = (dataset.groupby(['transaction', 'product'])['product'].count().reset_index(name='Count'))

train_basket.head()

,transaction,product,Count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1


Exemplo de resultado:
```
transaction | Dark chocolate | Latte | Croissant | Oatmeal | scone | ...
1000_0      |      1         |  0    |   0       |   0     |  1    | ...
```

In [78]:
# Converter pro formato
my_basket = train_basket.pivot_table(
    index="transaction", # índice que vai ficar na extremidade esquerda da mesa
    columns="product", # demais colunas
    values="Count"
).fillna(0)  # converter NaN com 0

my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0


In [79]:
# Normalizar os dados em 0s e 1s
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

my_basket_sets = my_basket.map(encode_units)

my_basket_sets.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [80]:
# Frequência dos produtos
frequent_items = apriori(my_basket_sets,
                         min_support = 0.05, # excluir itens com baixa frequência (insuficiente para recomendação)
                         use_colnames=True)  # usar nomes de coluna
frequent_items.head(20)

c:\Users\aleju\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115646,(Almond Croissant)
1,0.388889,(Cappuccino)
2,0.191232,(Carmel syrup)
3,0.112623,(Chocolate Chip Biscotti)
4,0.135676,(Chocolate Croissant)
5,0.188964,(Chocolate syrup)
6,0.116024,(Cranberry Scone)
7,0.114135,(Croissant)
8,0.277022,(Dark chocolate)
9,0.209373,(Espresso shot)


### Regra de Associação

In [ ]:
# Tabela que mostra o antecedente e consequência e sua frequência
rules_basket = association_rules(
    frequent_items, # dados: frequência dos produtos
    metric="lift", # lift - probabilidade de produtos serem comprados juntos
    min_threshold=1 # limite mínimo
)

rules_basket.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,0.008314,1.024775,0.255319
1,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,0.008314,1.133333,0.176432
2,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,0.025409,1.115717,0.611791
3,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,0.025409,1.436567,0.500152
4,(Almond Croissant),(Latte),0.115646,0.382086,0.054422,0.470588,1.231629,0.010235,1.167171,0.212660
5,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,0.010235,1.031236,0.304358
6,(Cappuccino),(Carmel syrup),0.388889,0.191232,0.102797,0.264334,1.382270,0.028429,1.099369,0.452540
7,(Carmel syrup),(Cappuccino),0.191232,0.388889,0.102797,0.537549,1.382270,0.028429,1.321462,0.341943
8,(Cappuccino),(Chocolate Chip Biscotti),0.388889,0.112623,0.056311,0.144801,1.285714,0.012514,1.037626,0.363636
9,(Chocolate Chip Biscotti),(Cappuccino),0.112623,0.388889,0.056311,0.500000,1.285714,0.012514,1.222222,0.250426


In [87]:
# Exemplo: produtos que são comprados juntos com Latte
rules_basket[
    rules_basket['antecedents'] == {'Latte'}
    ].sort_values('confidence', ascending=False) # ordenar pelo nível de confiança

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
71,(Latte),(Sugar Free Vanilla syrup),0.382086,0.200302,0.108844,0.284866,1.422182,0.032311,1.118249,0.480415
32,(Latte),(Carmel syrup),0.382086,0.191232,0.107710,0.281899,1.474121,0.034643,1.126260,0.520509
44,(Latte),(Chocolate syrup),0.382086,0.188964,0.103175,0.270030,1.428997,0.030974,1.111053,0.485842
69,(Latte),(Hazelnut syrup),0.382086,0.171580,0.101285,0.265084,1.544961,0.035727,1.127231,0.570848
53,(Latte),(Croissant),0.382086,0.114135,0.057067,0.149357,1.308605,0.013458,1.041407,0.381651
66,(Latte),(Ginger Scone),0.382086,0.133409,0.055178,0.144411,1.082472,0.004204,1.012860,0.123300
39,(Latte),(Chocolate Croissant),0.382086,0.135676,0.055178,0.144411,1.064381,0.003338,1.010209,0.097889
5,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,0.010235,1.031236,0.304358
65,(Latte),(Ginger Biscotti),0.382086,0.106198,0.054044,0.141444,1.331890,0.013467,1.041053,0.403272
48,(Latte),(Cranberry Scone),0.382086,0.116024,0.051398,0.134520,1.159416,0.007067,1.021371,0.222518


*Análise:* Quem compra Latte tem maior chance de comprar um Sugar Free Vanilla Syrup ou Carmel syrup

#### Salvar em JSON

In [ ]:
# Converter para JSON